In [2]:

pip install elasticsearch


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/524.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/524.6 kB ? eta -:--:--
   ---------------------------------------- 524.6/524.6 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install elasticsearch


In [ ]:
docker run -d --name elasticsearch -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.10.0


In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch()

# Create index
es.indices.create(index='employees', ignore=400)


In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('D:\Employee Sample Data 1.csv')

In [ ]:


# Index data
for index, document in df.iterrows():
    es.index(index='employees', id=index, document=document.to_dict())


In [ ]:
from flask import Flask, request, render_template
from elasticsearch import Elasticsearch

app = Flask(__name__)
es = Elasticsearch()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/search', methods=['POST'])
def search():
    name = request.form['name']
    results = es.search(index='employees', query={'match': {'name': name}})
    return render_template('results.html', results=results['hits']['hits'])

if __name__ == '__main__':
    app.run(debug=True)


In [10]:
def createCollection(p_collection_name):
    es.indices.create(index=p_collection_name, ignore=400)


In [11]:
def indexData(p_collection_name, p_exclude_column):
    # Load employee data
    df = pd.read_csv('employee_data.csv')
    
    # Exclude specified column
    df = df.drop(columns=[p_exclude_column])
    
    # Index data
    for index, row in df.iterrows():
        es.index(index=p_collection_name, id=index, document=row.to_dict())


In [12]:
def searchByColumn(p_collection_name, p_column_name, p_column_value):
    search_query = {"match": {p_column_name: p_column_value}}
    results = es.search(index=p_collection_name, query=search_query)
    return results['hits']['hits']


In [13]:
def delEmpById(p_collection_name, p_employee_id):
    es.delete(index=p_collection_name, id=p_employee_id)


In [14]:
def getDepFacet(p_collection_name):
    aggs_query = {
        "aggs": {
            "departments": {
                "terms": {"field": "Department.keyword"}
            }
        }
    }
    results = es.search(index=p_collection_name, body=aggs_query)
    return results['aggregations']['departments']['buckets']


In [ ]:
v_nameCollection = 'Hash_YourName'
v_phoneCollection = 'Hash_LastFourDigits'

# Create collections
createCollection(v_nameCollection)
createCollection(v_phoneCollection)

# Get employee count
print(getEmpCount(v_nameCollection))

# Index data
indexData(v_nameCollection, 'Department')
indexData(v_phoneCollection, 'Gender')

# Delete employee by ID
delEmpById(v_nameCollection, 'E02003')

# Get employee count again
print(getEmpCount(v_nameCollection))

# Search by column
print(searchByColumn(v_nameCollection, 'Department', 'IT'))
print(searchByColumn(v_nameCollection, 'Gender', 'Male'))
print(searchByColumn(v_phoneCollection, 'Department', 'IT'))

# Get department facet
print(getDepFacet(v_nameCollection))
print(getDepFacet(v_phoneCollection))
